# Create OMMX Adapters

OMMX is designed to exchange instance between modelers and solvers. This means each modeler needs to export their model into an [ommx.v1.Instance] and each solver needs to load an [ommx.v1.Instance] to solve it. We hope existing modelers and solvers implement their own exporter and loader of [ommx.v1.Instance], but it is possible to implement external libraries to translate the modeler-specific model between [ommx.v1.Instance]. Such libraries are called OMMX "Adapter".

[ommx-python-mip-adapeter](https://pypi.org/project/ommx-python-mip-adapter/) is an example of OMMX Adapter for [Python-MIP](https://www.python-mip.com/). This library translates the [Python-MIP's Model](https://python-mip.readthedocs.io/en/latest/classes.html#model) to [ommx.v1.Instance] and vice versa.

This notebook shows how to create an OMMX Adapter for a modeler-specific model.

[ommx.v1.Instance]: https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/index.html#ommx.v1.Instance

## Raw interface
OMMX Python SDK provides a two level interface, raw interface and high-level interface.

- The high-level interface provides a user-friendly interface for users who using OMMX-supported modelers and solvers.
- The raw interface provides a low-level interface for users who want to implement their own OMMX Adapter. This exposes the interface to protocol buffer messages directly.

This notebook uses the raw interface to create an OMMX Adapter.

In [1]:
from ommx.artifact import Artifact

In [2]:
artifact = Artifact.load_archive("../data/random_lp_instance.ommx")